# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import time
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.ops.gen_array_ops import shape
import random
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Model

In [3]:
def Def_model(X_test):
  input = tf.keras.Input(shape=X_test[0].shape)
  # CNN
  x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(input)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(3, padding='same')(x)
  x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(3, padding='same')(x)
  x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(3, padding='same')(x)
  x = tf.keras.layers.Flatten()(x)

  output = tf.keras.layers.Dense(64)(x)
  output = tf.keras.layers.Dense(2)(output)
  # output = tf.keras.layers.Dropout(.5,input_shape=(5,))(output)
  output = tf.keras.activations.sigmoid(output)
  global_model = tf.keras.Model(inputs=[input], outputs=[output])
  return global_model

# Hàm đánh giá

In [4]:
def Evaluation_Function(global_model,X_test,y_test):
    y_predict = global_model.predict(X_test).argmax(1)
    y_true = np.array(y_test).argmax(1)
    acc = tf.keras.metrics.Accuracy()
    acc.update_state(y_true, y_predict)
    acc = acc.result().numpy()
    recall = tf.keras.metrics.Recall()
    recall.update_state(y_true, y_predict)
    recall = recall.result().numpy()
    precision = tf.keras.metrics.Precision()
    precision.update_state(y_true, y_predict)
    precision = precision.result().numpy()
    f_score =  2 * recall * precision / (recall + precision)
    return [acc,recall,precision,f_score]

# Load data

In [5]:
def load_data(path, test = True,r = 1, agents = 1):
    if test:
        X = pd.read_csv(path)
    else:
        path = path + "PC" + str(agents) + ".csv"
        X = pd.read_csv(path)

    Y = X.iloc[: , -1]
    X= X.iloc[: , :-1]
    num_records = X.shape[0]
    X = tf.constant(X.values[:,0:187])
    X = tf.reshape(X, (num_records, 187, 1))
    Y= tf.one_hot(Y.values, 2)
    Y = tf.reshape(Y, (num_records, 2))
    return X, Y

    #   Y = X.iloc[:,-1:]
    # num_records = Y.shape[0]
    # X = tf.constant(X.values[:,0:30])
    # X = tf.reshape(X, (num_records, 30, 1))
    # Y = tf.one_hot(Y.values[:,0:], 2)
    # Y = tf.reshape(Y, (num_records, 2))
    # return X,Y

In [55]:
path = "/content/drive/MyDrive/SwarmLearn/Cardio/Flip_data/PC3.csv"
X_test, y_test = load_data(path)

In [7]:
from keras.optimizers import SGD
from sklearn.metrics import precision_score, recall_score, f1_score
global_model = Def_model(X_test)
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01)
global_model.compile(loss = 'binary_crossentropy', optimizer=optimizer, metrics=["accuracy"]
)

# Dữ liệu bình thường

In [9]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    st = time.time()
    list_agents_model = []
    global_weights = global_model.get_weights()
    avg_weights = []
    model_save = global_model.get_weights()
    for i in global_weights:
      avg_weights.append(np.zeros_like(i))
    # encryptweightlayer(global_weights)
    sum_data_train = 0
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(tf.keras.models.clone_model(global_model))
        list_agents_model[i].set_weights(global_weights)

    # ratio data
    for i in range(agents):
      path = "/content/drive/MyDrive/Khoa_luan/ECG_heatbeat/app-data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      sum_data_train += len(Y_train)
    # fed avg
    for i in range(agents):
      path = "/content/drive/MyDrive/Khoa_luan/ECG_heatbeat/app-data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 65, epochs=1) #

      local_weights = list_agents_model[i].get_weights()

      ratio = len(Y_train) / sum_data_train

      for j in range(len(local_weights)):
        avg_weights[j] += ratio * (local_weights[j] - model_save[j])

    global_model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=["accuracy"]
    )
    for i in range(len(global_weights)):
      avg_weights[i] = model_save[i] + avg_weights[i]
    global_model.set_weights(avg_weights)

  print('Final Evaluating:')
  [acc,recall,precision,f_score] = Evaluation_Function(global_model, X_test, y_test)
  return acc,recall,precision,f_score

In [10]:
FL_Train(rounds=5,agents=5,global_model=global_model)

Round  1
Local 0 Training:
452/452 [==============================] - 14s 28ms/step - loss: 0.2190 - accuracy: 0.9100
Local 1 Training:
452/452 [==============================] - 14s 28ms/step - loss: 0.2155 - accuracy: 0.9143
Local 2 Training:
452/452 [==============================] - 13s 27ms/step - loss: 0.2249 - accuracy: 0.9077
Local 3 Training:
452/452 [==============================] - 13s 25ms/step - loss: 0.2367 - accuracy: 0.8895
Local 4 Training:
452/452 [==============================] - 14s 28ms/step - loss: 0.2376 - accuracy: 0.8935
Round  2
Local 0 Training:
452/452 [==============================] - 13s 27ms/step - loss: 0.1436 - accuracy: 0.9514
Local 1 Training:
452/452 [==============================] - 13s 25ms/step - loss: 0.1443 - accuracy: 0.9522
Local 2 Training:
452/452 [==============================] - 13s 25ms/step - loss: 0.1487 - accuracy: 0.9510
Local 3 Training:
452/452 [==============================] - 13s 26ms/step - loss: 0.1424 - accuracy: 0.9519
L

(0.96890557, 0.850188, 0.96517634, 0.9040404003747775)

# TH2

In [36]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    list_agents_model = []
    avg_weights = []
    global_weights = global_model.get_weights()
    sum_data_train = 0
    for i in global_weights:
        avg_weights.append(np.zeros_like(i))
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(
            tf.keras.models.clone_model(global_model)
        )
        list_agents_model[i].set_weights(global_weights)
    model_save = global_model.get_weights()
    for i in range(agents):
      path = "/content/drive/MyDrive/Khoa_luan/ECG_flip/ECG_heatbeat_30/app-data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 128, epochs= 1,shuffle=False)

      local_weights = list_agents_model[i].get_weights()
      for j in range(len(local_weights)):
        avg_weights[j] += len(Y_train)* (local_weights[j] - model_save[j])
      sum_data_train += len(Y_train)
      for j in range(len(avg_weights)):
          avg_weights[j] = 0.6 * avg_weights[j] / sum_data_train
      global_model = tf.keras.models.clone_model(list_agents_model[0])
      global_model.compile(
              loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=["accuracy"]
          )
    for j in range(len(avg_weights)):
        avg_weights[j] = avg_weights[j] + model_save[j]
    global_model.set_weights(avg_weights)
  print('Final Evaluating:')
  acc,recall,precision,f_score = Evaluation_Function(global_model,X_test,y_test)
  return acc,recall,precision,f_score

In [37]:
FL_Train(rounds=5,agents=5,global_model=global_model)

Round  1
Local 0 Training:
230/230 [==============================] - 16s 59ms/step - loss: 0.0186 - accuracy: 0.9955
Local 1 Training:
230/230 [==============================] - 11s 42ms/step - loss: 0.0175 - accuracy: 0.9959
Local 2 Training:
230/230 [==============================] - 13s 44ms/step - loss: 0.0186 - accuracy: 0.9955
Local 3 Training:
230/230 [==============================] - 12s 45ms/step - loss: 0.0172 - accuracy: 0.9961
Local 4 Training:
235/235 [==============================] - 13s 47ms/step - loss: 0.0030 - accuracy: 0.9994
Round  2
Local 0 Training:
230/230 [==============================] - 13s 50ms/step - loss: 0.0182 - accuracy: 0.9956
Local 1 Training:
230/230 [==============================] - 13s 50ms/step - loss: 0.0168 - accuracy: 0.9962
Local 2 Training:
230/230 [==============================] - 13s 51ms/step - loss: 0.0182 - accuracy: 0.9956
Local 3 Training:
230/230 [==============================] - 13s 50ms/step - loss: 0.0168 - accuracy: 0.9963
L

(0.9387317, 0.6463487, 0.99694747, 0.7842477794121457)

# TH3

In [60]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    list_agents_model = []
    avg_weights = []
    global_weights = global_model.get_weights()
    sum_data_train = 0
    for i in global_weights:
        avg_weights.append(np.zeros_like(i))
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(
            tf.keras.models.clone_model(global_model)
        )
        list_agents_model[i].set_weights(global_weights)
    model_save = global_model.get_weights()
    for i in range(agents):
      path = "/content/drive/MyDrive/Khoa_luan/ECG_flip/ECG_heatbeat_30/app-data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 256, epochs= 2,shuffle=False)

      local_weights = list_agents_model[i].get_weights()
      for j in range(len(local_weights)):
        avg_weights[j] += len(Y_train)* (local_weights[j] - model_save[j])
      sum_data_train += len(Y_train)
      for j in range(len(avg_weights)):
          avg_weights[j] = 0.6 * avg_weights[j] / sum_data_train
      global_model = tf.keras.models.clone_model(list_agents_model[0])
      global_model.compile(
              loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=["accuracy"]
          )
    for j in range(len(avg_weights)):
        avg_weights[j] = avg_weights[j] + model_save[j]
    global_model.set_weights(avg_weights)
  print('Final Evaluating:')
  acc,recall,precision,f_score = Evaluation_Function(global_model,X_test,y_test)
  return acc,recall,precision,f_score

In [61]:
FL_Train(rounds=1,agents=5,global_model=global_model)

Round  1


FileNotFoundError: ignored